In [2]:
##  Installera bibliotek
import pickle
import os
import time
import google.generativeai as genai
import numpy as np
import json
import re
import nltk
import pandas as pd
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import pdfplumber
import streamlit as st
from dotenv import load_dotenv

c:\Users\CD\anaconda3\envs\Chatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CD\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Läs API-nyckeln
with open(".env", "r") as f:
    API_KEY = f.read().strip().split("=")[1].strip()

# Konfigurera API
genai.configure(api_key=API_KEY)

# Skapa modell
model = genai.GenerativeModel("models/gemini-2.0-flash")

print("*** ENKEL CHATBOT ***")
print("Skriv 'exit' för att avsluta")

while True:
    try:
        # Vänta på input
        message = input("\n> ")
        
        # Avsluta om användaren skriver exit
        if message.lower() == 'exit':
            break
            
        # Skicka meddelande till AI
        print("Väntar på svar...")
        response = model.generate_content(message)
        
        # Visa svaret
        print("\nAI:", response.text)
        
    except Exception as e:
        print(f"Ett fel inträffade: {e}")

print("Chatbot avslutad.")

*** ENKEL CHATBOT ***
Skriv 'exit' för att avsluta
Chatbot avslutad.


In [4]:
with open(".env", "r") as f:
    API_KEY = f.read().strip().split("=")[1].strip()

print(f"API_KEY är: {API_KEY[:5]}...")  # Visa bara början

API_KEY är: AIzaS...


In [60]:
import fitz  # pymupdf
import pandas as pd

def extract_and_save_text(pdf_path, output_text_file='extracted_text.txt'):
    # Öppna PDF-dokumentet
    doc = fitz.open(pdf_path)
    
    all_text = ""
    table_data = []
    
    # Gå igenom alla sidor
    for page_num in range(len(doc)):
        page = doc[page_num]
        
        # Extrahera text från sidan
        page_text = page.get_text()
        all_text += f"\n--- SIDA {page_num + 1} ---\n"
        all_text += page_text
        
        # Extrahera tabeller
        try:
            tables = page.find_tables()
            for i, table in enumerate(tables):
                table_extract = table.extract()
                df = pd.DataFrame(table_extract)
                table_data.append(df)
                
                # Lägg till tabelldata i texten också
                all_text += f"\n\n--- TABELL {i+1} PÅ SIDA {page_num + 1} ---\n"
                all_text += df.to_string(index=False)
                
        except Exception as e:
            print(f"Kunde inte extrahera tabeller från sida {page_num + 1}: {e}")
    
    # Stäng dokumentet
    doc.close()
    
    # Spara all text till fil
    with open(output_text_file, 'w', encoding='utf-8') as f:
        f.write(all_text)
    
    print(f"Text sparad till: {output_text_file}")
    print(f"Antal tabeller hittade: {len(table_data)}")
    
    return all_text, table_data

# Användning
pdf_path = "Visita_Branschriktlinjer-print_2021.pdf"
text, tables = extract_and_save_text(pdf_path, 'min_extraherade_text.txt')

# Visa första 500 tecken för att kontrollera
print("Första delen av extraherad text:")
print(text[:500])

Text sparad till: min_extraherade_text.txt
Antal tabeller hittade: 10
Första delen av extraherad text:

--- SIDA 1 ---
Visitas bransch- 
riktlinjer för  
restauranger
En beskrivning av hur livsmedelslagstiftningens krav på restaurang kan uppfyllas
Denna riktlinje upprättades 2014 
och uppdaterades  2021 04 16

--- SIDA 2 ---

--- SIDA 3 ---
15	
Visitas branschriktlinjer för livsmedelshantering på restaurang
17	
Dokumentation
18	
Faroanalys
12	
Utbildning och personalens kunskaper
16	
Personlig hygien
20	
Inköp
24	
Mottagning av varor
27	
Förvaring (torrt, kylt, fryst)
32	
Matlagning
39	
Varmhålln


In [61]:
import nltk
from nltk.tokenize import PunktSentenceTokenizer

# Träna en ny tokenizer baserat på din PDF-text
tokenizer = PunktSentenceTokenizer()
tokenizer.train(text)

# Dela upp texten i meningar
sentences = tokenizer.tokenize(text)

print(f"Antal meningar: {len(sentences)}")

Antal meningar: 2112


Semantic Chunking

In [62]:
# 1. FÖRBÄTTRAD CHUNKING FUNKTION
import re
import pickle
import nltk
nltk.download('punkt_tab')

def robust_pdf_chunking(text, target_size=500, overlap_sentences=1):
    """
    Robust chunking som hanterar PDF-text utan tydliga styckebrytningar
    """
    # Rensa och normalisera text
    text = re.sub(r'\r\n', '\n', text)
    text = re.sub(r'\n{3,}', '\n\n', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Dela upp i meningar med NLTK
    from nltk.tokenize import sent_tokenize
    sentences = sent_tokenize(text)
    
    chunks = []
    current_chunk = ""
    current_sentences = []
    
    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue
            
        # Testa om vi kan lägga till meningen
        test_chunk = current_chunk + " " + sentence if current_chunk else sentence
        
        if len(test_chunk) <= target_size:
            # Lägg till meningen
            current_chunk = test_chunk
            current_sentences.append(sentence)
        else:
            # Spara nuvarande chunk om den inte är tom
            if current_chunk:
                chunks.append(current_chunk.strip())
                
                # Skapa överlappning
                if overlap_sentences > 0 and len(current_sentences) > overlap_sentences:
                    overlap_sents = current_sentences[-overlap_sentences:]
                    current_chunk = " ".join(overlap_sents) + " " + sentence
                    current_sentences = overlap_sents + [sentence]
                else:
                    current_chunk = sentence
                    current_sentences = [sentence]
            else:
                # Om första meningen är för lång, lägg till den ändå
                current_chunk = sentence
                current_sentences = [sentence]
    
    # Lägg till sista chunken
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

# 2. SKAPA CHUNKS
print("Skapar chunks...")
chunks = robust_pdf_chunking(text, target_size=500, overlap_sentences=1)

print(f"✅ {len(chunks)} chunks skapade")

# Spara chunks
with open("semantic_chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)

Skapar chunks...
✅ 862 chunks skapade


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\CD\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Skapar embeddings

In [63]:
# SKAPA EMBEDDINGS FUNKTION
def create_embeddings(text):
    """
    Skapar embeddings med Gemini API
    """
    try:
        result = genai.embed_content(
            model="models/text-embedding-004",
            content=text,
            task_type="retrieval_document"
        )
        return {"embedding": result['embedding']}
    except Exception as e:
        print(f"Fel vid skapande av embedding: {e}")
        return None
    
# SKAPA EMBEDDINGS FÖR ALLA CHUNKS
print(f"\nSkapar embeddings för {len(chunks)} chunks...")
embeddings = []

for i, chunk in enumerate(chunks):
    print(f"Bearbetar chunk {i+1}/{len(chunks)}")
    
    embedding_result = create_embeddings(chunk)
    if embedding_result:
        embeddings.append(embedding_result["embedding"])
    else:
        print(f"Misslyckades med chunk {i+1}")
        
    # Lägg till en liten paus för att undvika rate limiting
    time.sleep(0.1)

print(f"✅ {len(embeddings)} embeddings skapade")

# Spara embeddings
with open("embeddings_chunks.pkl", "wb") as f:
    pickle.dump(embeddings, f)


Skapar embeddings för 862 chunks...
Bearbetar chunk 1/862
Bearbetar chunk 2/862
Bearbetar chunk 3/862
Bearbetar chunk 4/862
Bearbetar chunk 5/862
Bearbetar chunk 6/862
Bearbetar chunk 7/862
Bearbetar chunk 8/862
Bearbetar chunk 9/862
Bearbetar chunk 10/862
Bearbetar chunk 11/862
Bearbetar chunk 12/862
Bearbetar chunk 13/862
Bearbetar chunk 14/862
Bearbetar chunk 15/862
Bearbetar chunk 16/862
Bearbetar chunk 17/862
Bearbetar chunk 18/862
Bearbetar chunk 19/862
Bearbetar chunk 20/862
Bearbetar chunk 21/862
Bearbetar chunk 22/862
Bearbetar chunk 23/862
Bearbetar chunk 24/862
Bearbetar chunk 25/862
Bearbetar chunk 26/862
Bearbetar chunk 27/862
Bearbetar chunk 28/862
Bearbetar chunk 29/862
Bearbetar chunk 30/862
Bearbetar chunk 31/862
Bearbetar chunk 32/862
Bearbetar chunk 33/862
Bearbetar chunk 34/862
Bearbetar chunk 35/862
Bearbetar chunk 36/862
Bearbetar chunk 37/862
Bearbetar chunk 38/862
Bearbetar chunk 39/862
Bearbetar chunk 40/862
Bearbetar chunk 41/862
Bearbetar chunk 42/862
Bearbe

Semantic search

In [64]:
import logging
from typing import List, Any
import numpy as np

# Sätt upp grundläggande logging-konfiguration
logging.basicConfig(level=logging.INFO)

# UPPDATERAD COSINE SIMILARITY FUNKTION
def cosine_similarity(vec1, vec2):
    """
    Beräknar cosine similarity mellan två vektorer
    """
    # Konvertera till numpy arrays om det behövs
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    
    # Beräkna cosine similarity
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    
    if norm1 == 0 or norm2 == 0:
        return 0
    
    return dot_product / (norm1 * norm2)

def semantic_search(query: str, chunks: List[str], embeddings: List[List[float]], k: int = 5) -> List[str]:
    """
    Robust semantic search-funktion med felhantering, typkontroller och loggning.
    
    Parameters:
    - query (str): Frågan som användaren söker efter
    - chunks (List[str]): Lista med textdelar att söka i
    - embeddings (List[List[float]]): Lista med embeddings för varje chunk
    - k (int): Antal resultat att returnera

    Returns:
    - List[str]: De k mest relevanta textdelarna
    """
    try:
        if not isinstance(query, str):
            raise ValueError("Query måste vara en sträng.")
        if not isinstance(chunks, list) or not all(isinstance(c, str) for c in chunks):
            raise ValueError("Chunks måste vara en lista med strängar.")
        if not isinstance(embeddings, list) or not all(isinstance(e, list) for e in embeddings):
            raise ValueError("Embeddings måste vara en lista med listor (floats).")
        if len(chunks) != len(embeddings):
            raise ValueError("Antalet chunks och embeddings måste vara lika.")

        # Skapa embedding för frågan
        query_result = create_embeddings(query)
        if not query_result or "embedding" not in query_result:
            logging.warning("Kunde inte skapa embedding för frågan.")
            return chunks[:k]  # Fallback

        query_embedding = query_result["embedding"]

        similarity_scores = []

        # Beräkna cosine similarity mellan frågan och varje chunk
        for i, chunk_embedding in enumerate(embeddings):
            try:
                similarity = cosine_similarity(query_embedding, chunk_embedding)
                similarity_scores.append((i, similarity))
            except Exception as e:
                logging.warning(f"Fel vid beräkning av similarity för chunk {i}: {e}")
                similarity_scores.append((i, 0))  # Sätt similarity till 0 vid fel

        # Sortera efter högst likhet
        similarity_scores.sort(key=lambda x: x[1], reverse=True)

        # Plocka ut de k bästa
        top_indices = [index for index, _ in similarity_scores[:k]]

        return [chunks[i] for i in top_indices]

    except Exception as e:
        logging.error(f"Fel i semantic_search: {e}")
        return chunks[:k]  # Fallback

Respons Generering

In [65]:
def generate_response(system_prompt, user_message):
    """
    Genererar svar baserat på RAG
    """
    try:
        # Hitta relevanta chunks
        relevant_chunks = semantic_search(user_message, chunks, embeddings)
        
        # Skapa kontext
        context = "\n\n".join(relevant_chunks)
        
        # Skapa user prompt
        user_prompt = f"Frågan är: {user_message}\n\nHär är kontexten:\n{context}"
        
        # Kombinera system och user prompt
        full_prompt = f"{system_prompt}\n\n{user_prompt}"
        
        # Generera svar
        response = model.generate_content(full_prompt)
        return response
        
    except Exception as e:
        print(f"Fel vid generering av svar: {e}")
        return None

Systempromt

In [66]:
system_prompt = """
Svara alltid med max 2 hela meningar. Avsluta alltid svaret med en punkt. Om du når tokengränsen, avsluta med en fullständig mening, även om det innebär att du måste korta svaret något. 
Använd enkel svenska utan listor eller markdown. Lägg till frågan: 'Vill du ha ett mer detaljerat svar?' sist.

Exempel:
Fråga: Vad är HACCP?
Svar: HACCP är en metod för att identifiera faror i livsmedel. Den minskar risken för sjukdom. Vill du ha ett mer detaljerat svar?

Fråga: {question}
Svar:"""

Testar

In [67]:
print("\n" + "="*50)
print("TESTAR RAG-SYSTEMET")
print("="*50)

# Testa med en fråga
test_query = "Vad är en faroanalys?"
print(f"\nFråga: {test_query}")

response = generate_response(system_prompt, test_query)
if response:
    print("\nSvar:")
    print(response.text)
else:
    print("Kunde inte generera svar")


TESTAR RAG-SYSTEMET

Fråga: Vad är en faroanalys?

Svar:
En faroanalys identifierar och bedömer potentiella faror i livsmedelshanteringen. Den hjälper till att förebygga risker för konsumenterna. Vill du ha ett mer detaljerat svar?



In [69]:
from numpy import dot
from numpy.linalg import norm
import pandas as pd
import json

# Funktion för cosine similarity med embeddings från genai
def cosine_similarity(vec1, vec2):
    dot_product = dot(vec1, vec2)
    norm1 = norm(vec1)
    norm2 = norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return dot_product / (norm1 * norm2)

def calculate_cosine_similarity(text1, text2, model="models/embedding-001"):
    embedding1 = genai.embed_content(content=text1, model=model, task_type="semantic_similarity")["embedding"]
    embedding2 = genai.embed_content(content=text2, model=model, task_type="semantic_similarity")["embedding"]
    return cosine_similarity(embedding1, embedding2)

# Faithfulness-prompt
FAITHFULNESS_PROMPT = """
Bedöm hur faktamässigt korrekt AI:s svar är jämfört med det önskade svaret.
Svara endast med ett tal mellan 0 och 1.

Fråga: {question}
Önskat svar: {true_answer}
AI:s svar: {response}

Poäng:
"""

# Relevancy-prompt
RELEVANCY_PROMPT = """
Bedöm hur relevant AI:s svar är i förhållande till frågan.
Svara endast med ett tal mellan 0 och 1.

Fråga: {question}
AI:s svar: {response}

Poäng:
"""

# Läs in valideringsdata
with open("validation_data.json", "r", encoding="utf-8") as f:
    validation_data = json.load(f)

results = []

for i, item in enumerate(validation_data, 1):
    question = item["question"]
    ideal_answer = item["ideal_answer"]

    try:
        prompt = f"{system_prompt}\n\nFråga: {question}\nSvar:"
        response = model.generate_content(
            prompt,
            generation_config={"max_output_tokens": 100}
        )
        model_answer = response.text.strip()

        result = {
            "question": question,
            "ideal_answer": ideal_answer,
            "model_answer": model_answer
        }

        # Faithfulness
        prompt_f = FAITHFULNESS_PROMPT.format(question=question, true_answer=ideal_answer, response=model_answer)
        try:
            resp_f = model.generate_content(prompt_f)
            result["faithfulness"] = max(0.0, min(1.0, float(resp_f.text.strip())))
        except Exception as e:
            print(f"[{i}] Fel vid faithfulness: {e}")
            result["faithfulness"] = 0.0

        # Relevancy
        prompt_r = RELEVANCY_PROMPT.format(question=question, response=model_answer)
        try:
            resp_r = model.generate_content(prompt_r)
            result["relevancy"] = max(0.0, min(1.0, float(resp_r.text.strip())))
        except Exception as e:
            print(f"[{i}] Fel vid relevancy: {e}")
            result["relevancy"] = 0.0

        # Similarity
        try:
            result["similarity_score"] = calculate_cosine_similarity(model_answer, ideal_answer)
        except Exception as e:
            print(f"[{i}] Fel vid similarity: {e}")
            result["similarity_score"] = 0.0

        # Medelpoäng
        result["avg_score"] = round((result["faithfulness"] + result["relevancy"] + result["similarity_score"]) / 3, 4)

        print(f"[{i}] Avg Score: {result['avg_score']:.3f}")
        results.append(result)

    except Exception as main_e:
        print(f"[{i}] Fel vid fråga: {question}\n{main_e}")
        continue

# Skapa dataframe
df = pd.DataFrame(results)

# Skriv sammanställning till terminal
print("\n Sammanställning av individuella poäng:")
display_cols = ["question", "faithfulness", "relevancy", "similarity_score", "avg_score"]
df_display = df[display_cols]
overall_avg = round(df_display["avg_score"].mean(), 4)

summary_row = {
    "question": "GENOMSNITT:",
    "faithfulness": round(df_display["faithfulness"].mean(), 2),
    "relevancy": round(df_display["relevancy"].mean(), 2),
    "similarity_score": round(df_display["similarity_score"].mean(), 2),
    "avg_score": overall_avg
}

df_display = pd.concat([df_display, pd.DataFrame([summary_row])], ignore_index=True)
print(df_display.to_string(index=False))

# Skapa och spara formaterad utvärdering
formatted_output = []

for i, row in df.iterrows():
    entry = f"""
============================================================
Fråga {i+1}:
{row['question']}

AI:s svar:
{row['model_answer']}

Poäng:
- Faktatrohet (Faithfulness): {row['faithfulness']:.2f}
- Relevans (Relevancy): {row['relevancy']:.2f}
- Likhet med idealsvar (Similarity): {row['similarity_score']:.2f}
- Medelpoäng: {row['avg_score']:.2f}
"""
    formatted_output.append(entry.strip())

# Lägg till summerad poäng
summary_entry = f"""
============================================================
SAMMANLAGD MEDELPOÄNG:

- Genomsnittlig poäng över alla frågor: {overall_avg:.2f}
"""
formatted_output.append(summary_entry.strip())

# Spara som JSON
with open("komponent_utvardering.json", "w", encoding="utf-8") as f_json:
    json.dump(formatted_output, f_json, ensure_ascii=False, indent=2)

# Spara som CSV
df_formatted = pd.DataFrame({"utvardering": formatted_output})
df_formatted.to_csv("komponent_utvardering.csv", index=False, encoding="utf-8-sig")

print("\n✅ Formaterad utvärdering sparad som JSON och CSV.")


[1] Avg Score: 0.979
[2] Avg Score: 0.792
[3] Avg Score: 0.835
[4] Avg Score: 0.742
[5] Avg Score: 0.754
[6] Avg Score: 0.716

 Sammanställning av individuella poäng:
                                                                               question  faithfulness  relevancy  similarity_score  avg_score
                      Vad är HACCP och varför är det viktigt i en restaurangverksamhet?          1.00       1.00          0.937392     0.9791
                                Vad menas med intern spårbarhet och är det ett lagkrav?          0.50       1.00          0.876225     0.7921
                          Vilken information måste gäster kunna få om maten vid frågor?          0.60       1.00          0.904978     0.8350
Vad ska göras om det uppstår problem med förpackningar eller temperaturer vid leverans?          0.70       0.70          0.826681     0.7422
                                             Hur ska man dokumentera en genomförd kurs?          0.60       0.90          0

## Utvärdering

In [68]:
from numpy import dot
from numpy.linalg import norm
import pandas as pd
import json

# Funktion för cosine similarity med embeddings från genai
def cosine_similarity(vec1, vec2):
    dot_product = dot(vec1, vec2)
    norm1 = norm(vec1)
    norm2 = norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return 0.0
    return dot_product / (norm1 * norm2)

def calculate_cosine_similarity(text1, text2, model="models/embedding-001"):
    embedding1 = genai.embed_content(content=text1, model=model, task_type="semantic_similarity")["embedding"]
    embedding2 = genai.embed_content(content=text2, model=model, task_type="semantic_similarity")["embedding"]
    return cosine_similarity(embedding1, embedding2)

# Faithfulness-prompt
FAITHFULNESS_PROMPT = """
Bedöm hur faktamässigt korrekt AI:s svar är jämfört med det önskade svaret.
Svara endast med ett tal mellan 0 och 1.

Fråga: {question}
Önskat svar: {true_answer}
AI:s svar: {response}

Poäng:
"""

# Relevancy-prompt
RELEVANCY_PROMPT = """
Bedöm hur relevant AI:s svar är i förhållande till frågan.
Svara endast med ett tal mellan 0 och 1.

Fråga: {question}
AI:s svar: {response}

Poäng:
"""

# Läs in valideringsdata
with open("validation_data.json", "r", encoding="utf-8") as f:
    validation_data = json.load(f)

results = []

for i, item in enumerate(validation_data, 1):
    question = item["question"]
    ideal_answer = item["ideal_answer"]

    try:
         # 1. Generera AI-svar med systemprompt och tokenbegränsning
        prompt = f"{system_prompt}\n\nFråga: {question}\nSvar:"
        response = model.generate_content(
        prompt,
        generation_config={"max_output_tokens": 100}
        )
        model_answer = response.text.strip()

        result = {
        "question": question,
        "ideal_answer": ideal_answer,
        "model_answer": model_answer
        }


        # 2. Faithfulness
        prompt_f = FAITHFULNESS_PROMPT.format(question=question, true_answer=ideal_answer, response=model_answer)
        try:
            resp_f = model.generate_content(prompt_f)
            result["faithfulness"] = max(0.0, min(1.0, float(resp_f.text.strip())))
        except Exception as e:
            print(f"[{i}] Fel vid faithfulness: {e}")
            result["faithfulness"] = 0.0

        # 3. Relevancy
        prompt_r = RELEVANCY_PROMPT.format(question=question, response=model_answer)
        try:
            resp_r = model.generate_content(prompt_r)
            result["relevancy"] = max(0.0, min(1.0, float(resp_r.text.strip())))
        except Exception as e:
            print(f"[{i}] Fel vid relevancy: {e}")
            result["relevancy"] = 0.0

        # 4. Similarity
        try:
            result["similarity_score"] = calculate_cosine_similarity(model_answer, ideal_answer)
        except Exception as e:
            print(f"[{i}] Fel vid similarity: {e}")
            result["similarity_score"] = 0.0

        # 5. Medelpoäng
        result["avg_score"] = round((
            result["faithfulness"] +
            result["relevancy"] +
            result["similarity_score"]
        ) / 3, 4)

        print(f"[{i}] Avg Score: {result['avg_score']:.3f}")
        results.append(result)

    except Exception as main_e:
        print(f"[{i}] Fel vid fråga: {question}\n{main_e}")
        continue

# Lägg till sammanlagd medelpoäng som sista rad i tabellen

# Skapa dataframe för resultat
print("\n Sammanställning av individuella poäng:")
display_cols = ["question", "faithfulness", "relevancy", "similarity_score", "avg_score"]
df_display = pd.DataFrame(results)[display_cols]

overall_avg = round(df_display["avg_score"].mean(), 4)
summary_row = {
    "question": "GENOMSNITT:",
    "faithfulness": round(df_display["faithfulness"].mean(), 2),
    "relevancy": round(df_display["relevancy"].mean(), 2),
    "similarity_score": round(df_display["similarity_score"].mean(), 2),
    "avg_score": overall_avg
}

df = pd.DataFrame(results)

# Spara strukturerade resultatfiler direkt efter bearbetning
with open("komponent_utvardering_rådata.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)
df.to_csv("komponent_utvardering_rådata.csv", index=False, encoding="utf-8-sig")


df_display = pd.concat([df_display, pd.DataFrame([summary_row])], ignore_index=True)
print(df_display.to_string(index=False))
print(" Utvärdering klar")

[1] Avg Score: 0.880
[2] Avg Score: 0.785
[3] Avg Score: 0.840
[4] Avg Score: 0.819
[5] Avg Score: 0.782
[6] Avg Score: 0.718

 Sammanställning av individuella poäng:
                                                                               question  faithfulness  relevancy  similarity_score  avg_score
                      Vad är HACCP och varför är det viktigt i en restaurangverksamhet?          0.70       1.00          0.939741     0.8799
                                Vad menas med intern spårbarhet och är det ett lagkrav?          0.50       1.00          0.854792     0.7849
                          Vilken information måste gäster kunna få om maten vid frågor?          0.60       1.00          0.918353     0.8395
Vad ska göras om det uppstår problem med förpackningar eller temperaturer vid leverans?          0.70       0.90          0.856158     0.8187
                                             Hur ska man dokumentera en genomförd kurs?          0.70       0.80          0

Omvandlar chatbot till Streamlit-format

In [ ]:
# Läs in API-nyckel från .env
load_dotenv()
API_KEY = os.getenv("API_KEY")
genai.configure(api_key=API_KEY)

# Skapa modell
model = genai.GenerativeModel("models/gemini-2.0-flash")

st.title("🧠 Enkel Chatbot med Gemini")
st.write("Ställ en fråga i rutan nedan:")

# Användarens fråga
user_input = st.text_input("Fråga", "")

if user_input:
    try:
        response = model.generate_content(user_input)
        st.subheader("💬 Svar:")
        st.write(response.text)
    except Exception as e:
        st.error(f"Något gick fel: {e}")